In [1]:
################################################################### new code #################################################################
import pandas as pd 
import talib
import numpy as np
df = pd.read_excel("full data.xlsx")

In [2]:
df

,Ticker,Date/Time,Open,High,Low,Close,Volume
0,ADANIPORTS-I,2020-06-08 09:29:59,340.75,347.15,340.75,345.75,547500
1,ADANIPORTS-I,2020-06-08 09:44:59,345.60,346.05,344.00,345.20,442500
2,ADANIPORTS-I,2020-06-08 09:59:59,345.20,345.35,342.20,342.35,215000
3,ADANIPORTS-I,2020-06-08 10:14:59,342.45,343.45,342.40,343.30,130000
4,ADANIPORTS-I,2020-06-08 10:29:59,343.30,343.30,342.00,342.65,162500
...,...,...,...,...,...,...,...
5707,ADANIPORTS-I,2021-05-05 14:29:59,742.75,745.55,742.20,744.70,295000
5708,ADANIPORTS-I,2021-05-05 14:44:59,744.70,745.45,742.45,742.75,505000
5709,ADANIPORTS-I,2021-05-05 14:59:59,742.75,743.60,738.35,739.00,1017500
5710,ADANIPORTS-I,2021-05-05 15:14:59,739.00,742.30,738.00,742.00,1190000


In [3]:
df['Ema1'] = talib.EMA(df['Close'], timeperiod=20)
df['Ema2'] = talib.EMA(df['Close'], timeperiod=50)
df['RSI1'] = talib.RSI(df['Close'], timeperiod=14)

# Define entry and exit signals
# Define Buy and Sell Signals
df['Buy_Signal'] = (df['Ema1'] > df['Ema2']) & (df['Ema1'].shift(1) <= df['Ema2'].shift(1)) & (df['RSI1'] > 60)
df['Sell_Signal'] = (df['Ema1'] < df['Ema2']) & (df['Ema1'].shift(1) >= df['Ema2'].shift(1)) & (df['RSI1'] < 40)


In [4]:
df

,Ticker,Date/Time,Open,High,Low,Close,Volume,Ema1,Ema2,RSI1,Buy_Signal,Sell_Signal
0,ADANIPORTS-I,2020-06-08 09:29:59,340.75,347.15,340.75,345.75,547500,NaN,NaN,NaN,False,False
1,ADANIPORTS-I,2020-06-08 09:44:59,345.60,346.05,344.00,345.20,442500,NaN,NaN,NaN,False,False
2,ADANIPORTS-I,2020-06-08 09:59:59,345.20,345.35,342.20,342.35,215000,NaN,NaN,NaN,False,False
3,ADANIPORTS-I,2020-06-08 10:14:59,342.45,343.45,342.40,343.30,130000,NaN,NaN,NaN,False,False
4,ADANIPORTS-I,2020-06-08 10:29:59,343.30,343.30,342.00,342.65,162500,NaN,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
5707,ADANIPORTS-I,2021-05-05 14:29:59,742.75,745.55,742.20,744.70,295000,748.143092,754.217024,40.771072,False,False
5708,ADANIPORTS-I,2021-05-05 14:44:59,744.70,745.45,742.45,742.75,505000,747.629464,753.767337,38.335038,False,False
5709,ADANIPORTS-I,2021-05-05 14:59:59,742.75,743.60,738.35,739.00,1017500,746.807611,753.188226,34.113775,False,False
5710,ADANIPORTS-I,2021-05-05 15:14:59,739.00,742.30,738.00,742.00,1190000,746.349743,752.749472,39.822694,False,False


In [5]:
df["datetime"]=df["Date/Time"]

In [6]:
df['Date'] = pd.to_datetime(df['datetime']).dt.date
df['Time'] = pd.to_datetime(df['datetime']).dt.time


In [7]:
################################## define the constants ########################################################
TARGET_PERCENTAGE = 0.05
STOPLOSS_PERCENTAGE = 0.02
TRAILING_STOP_PERCENTAGE = 0.03
# Define time range for entries
entry_start_time = pd.to_datetime('09:30:00').time()
entry_end_time = pd.to_datetime('14:45:00').time()

In [8]:

# Initialize the list and dictionary
trades_list = []
trade_details = {}

# Initialize the current position
current_position = None


for i in range(len(df)):
    # Time of the current row
    current_time = df.iloc[i].Time
    
    # Entry logic - only during specified time
    if current_position is None and entry_start_time <= current_time <= entry_end_time:
        # Check Entry condition
        if df.iloc[i]['Buy_Signal']:  # Long Entry
            trade_details = {
                'Signal': "Buy",
                'Entry Date': df.iloc[i].Date,
                'Entry Time': df.iloc[i].Time,
                'Entry Price': df.iloc[i].Close,
                'Target Price': df.iloc[i].Close * (1 + TARGET_PERCENTAGE),
                'Stop Loss Price': df.iloc[i].Close * (1 - STOPLOSS_PERCENTAGE),
                'Trailing Stop Price': df.iloc[i].Close * (1 - TRAILING_STOP_PERCENTAGE),
                'Exit Date': np.nan,
                'Exit Time': np.nan,
                'Exit Price': np.nan,
                'Hit Target/Stop Loss': np.nan
            }
            current_position = "Buy"
            
        elif df.iloc[i]['Sell_Signal']:  # Short Entry
            trade_details = {
                'Signal': "Sell",
                'Entry Date': df.iloc[i].Date,
                'Entry Time': df.iloc[i].Time,
                'Entry Price': df.iloc[i].Close,
                'Target Price': df.iloc[i].Close * (1 - TARGET_PERCENTAGE),
                'Stop Loss Price': df.iloc[i].Close * (1 + STOPLOSS_PERCENTAGE),
                'Trailing Stop Price': df.iloc[i].Close * (1 + TRAILING_STOP_PERCENTAGE),
                'Exit Date': np.nan,
                'Exit Time': np.nan,
                'Exit Price': np.nan,
                'Hit Target/Stop Loss': np.nan
            }
            current_position = "Sell"

    # Exit logic - can occur at any time during the trading day
    if current_position is not None:
        if current_position == 'Buy':  # Long position
            if df.iloc[i].Close > trade_details['Entry Price']:
                trade_details['Trailing Stop Price'] = max(trade_details['Trailing Stop Price'], df.iloc[i].Close * (1 - TRAILING_STOP_PERCENTAGE))
            
            if df.iloc[i].High >= trade_details['Target Price']:
                trade_details.update({
                    'Exit Date': df.iloc[i].Date,
                    'Exit Time': df.iloc[i].Time,
                    'Exit Price': trade_details['Target Price'],
                    'Hit Target/Stop Loss': 'Target'
                })
                trades_list.append(trade_details)
                trade_details = {}
                current_position = None

            elif df.iloc[i].Close <= trade_details['Stop Loss Price']:
                trade_details.update({
                    'Exit Date': df.iloc[i].Date,
                    'Exit Time': df.iloc[i].Time,
                    'Exit Price': trade_details['Stop Loss Price'],
                    'Hit Target/Stop Loss': 'Stop Loss'
                })
                trades_list.append(trade_details)
                trade_details = {}
                current_position = None

            elif df.iloc[i].Close <= trade_details['Trailing Stop Price']:
                trade_details.update({
                    'Exit Date': df.iloc[i].Date,
                    'Exit Time': df.iloc[i].Time,
                    'Exit Price': trade_details['Trailing Stop Price'],
                    'Hit Target/Stop Loss': 'Trailing Stop'
                })
                trades_list.append(trade_details)
                trade_details = {}
                current_position = None

        elif current_position == 'Sell':  # Short position
            if df.iloc[i].Close < trade_details['Entry Price']:
                trade_details['Trailing Stop Price'] = min(trade_details['Trailing Stop Price'], df.iloc[i].Close * (1 + TRAILING_STOP_PERCENTAGE))
            
            if df.iloc[i].Low <= trade_details['Target Price']:
                trade_details.update({
                    'Exit Date': df.iloc[i].Date,
                    'Exit Time': df.iloc[i].Time,
                    'Exit Price': trade_details['Target Price'],
                    'Hit Target/Stop Loss': 'Target'
                })
                trades_list.append(trade_details)
                trade_details = {}
                current_position = None

            elif df.iloc[i].Close >= trade_details['Stop Loss Price']:
                trade_details.update({
                    'Exit Date': df.iloc[i].Date,
                    'Exit Time': df.iloc[i].Time,
                    'Exit Price': trade_details['Stop Loss Price'],
                    'Hit Target/Stop Loss': 'Stop Loss'
                })
                trades_list.append(trade_details)
                trade_details = {}
                current_position = None

            elif df.iloc[i].Close >= trade_details['Trailing Stop Price']:
                trade_details.update({
                    'Exit Date': df.iloc[i].Date,
                    'Exit Time': df.iloc[i].Time,
                    'Exit Price': trade_details['Trailing Stop Price'],
                    'Hit Target/Stop Loss': 'Trailing Stop'
                })
                trades_list.append(trade_details)
                trade_details = {}
                current_position = None

# Save the results to an Excel file
trades_df = pd.DataFrame(trades_list)
trades_df.to_excel("trades_output.xlsx", index=False)

print("Trades saved to 'trades_output.xlsx'.")

Trades saved to 'trades_output.xlsx'.
